In [86]:
import numpy as np

class TicTacToe3D:
    def __init__(self):
        self.board = np.zeros((4, 4, 4), dtype=int)
        self.heights = np.zeros((4, 4), dtype=int)

    def move(self, x, y, player):
        # z = self.heights[x, y]
        # #print(x, y, z)
        # if z >= 4:
        #     return False
        # if self.board[x, y, z] != 0:
        #     return False
        # self.board[x, y, z] = self.turn
        # self.heights[x, y] += 1
        # return True
        if not (0 <= x < 4 and 0 <= y < 4):
            return False

        z = self.heights[y, x]
        if z >= 4:
            return False
        if self.board[z, y, x] != 0:
            return False

        self.board[z, y, x] = player
        self.heights[y, x] += 1

        return True

    def check(self):
        # check if any player has connected 4 in a row
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    if self.board[i, j, k] == 0:
                        continue
                    oneD = [(1,0,0), (0,1,0), (0,0,1)]
                    twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
                    threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
                    for dx, dy, dz in oneD + twoD + threeD:
                        for l in range(1, 4):
                            if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4 or self.board[i + dx * l, j + dy * l, k + dz * l] != self.board[i, j, k]:
                                break
                        else:
                            return self.board[i, j, k]
        return 0

    def print(self):
        for k in range(4):
            for i in range(4):
                for j in range(4):
                    print('OAB'[self.board[i, j, k]], end=' ')
                print()
            print()

    def loadState(self, board):
        self.board = board.copy()
        last_non_zero = np.argmax(board[::-1] != 0, axis=0)
        self.heights = np.where(last_non_zero == 0, 0, 4 - last_non_zero)


        # self.heights = np.zeros((4, 4), dtype=int)
        # for i in range(4):
        #     for j in range(4):
        #         for k in range(4):
        #             if self.board[i, j, k] != 0:
        #                 self.heights[i, j] = k + 1
        return self

In [87]:
# all possible wins masks
oneD = [(1,0,0), (0,1,0), (0,0,1)]
twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
win_masks = oneD + twoD + threeD

all_wins = []
tmp = np.zeros((4, 4, 4), dtype=int)
for i in range(4):
    for j in range(4):
        for k in range(4):
            for dx, dy, dz in win_masks:
                for l in range(4):
                    if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4:
                        break
                    tmp[i + dx * l, j + dy * l, k + dz * l] = 1
                else:
                    all_wins.append(tmp.copy())
                tmp.fill(0)

print(len(all_wins))

76


In [88]:
def main():
    game = TicTacToe3D()
    while True:
        print('\r')
        game.print()
        x = int(input('Enter x: '))
        y = int(input('Enter y: '))
        if not game.move(x, y):
            print('Invalid move')
        if (result := game.check()):
            game.print()
            print('Player', 'OAB'[result], 'wins!')
            break

In [89]:
def getHeightFromBoard(board):
    height = [[0 for _ in range(4)] for _ in range(4)]
    for i in range(4):
        for j in range(4):
            for k in range(4):
                if board[i, j, k] != 0:
                    height[i, j] = k + 1
    return height

In [90]:
def printBoard(board):
    for k in range(4):
        for i in range(4):
            for j in range(4):
                print('OAB'[board[i, j, k]], end=' ')
            print()
        print()

In [96]:
import random

# def count_wins(board, player):
#     c = 0
#     # check for 4 in a row for player
#     for i in range(4):
#         for j in range(4):
#             for k in range(4):
#                 if board[i, j, k] == player:
#                     oneD = [(1,0,0), (0,1,0), (0,0,1)]
#                     twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
#                     threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
#                     for dx, dy, dz in oneD + twoD + threeD:
#                         for l in range(1, 4):
#                             if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4 or board[i + dx * l, j + dy * l, k + dz * l] != player:
#                                 break
#                         else:
#                             c += 1
#     return c

def count_wins(board, player):
    global all_wins
    c = 0
    for win in all_wins:
        if np.sum(win * board) == 4 * player:
            c += 1
    return c

def heuristic(board, player, count_player_wins):
    opposing_filled = board.copy()
    opposing_filled[opposing_filled == 0] = -player
    return count_player_wins - count_wins(opposing_filled, -player)

def minimax(board, player, depth, alpha, beta, minimizingPlayer):
    game = TicTacToe3D().loadState(board)
    result = game.check()
    if result == player:
        return 1000 + depth
    elif result == -player:
        return -1000 - depth
    if depth == 0 or np.all(game.heights == 4):  # Check for a full board
        player_filled = board.copy()
        player_filled[player_filled == 0] = player
        return heuristic(board, player, count_wins(player_filled, player))

    if minimizingPlayer:
        value = float('-inf')
        for i in range(4):
            for j in range(4):
                if game.heights[i, j] < 4:
                    game.move(i, j, player)
                    value = max(value, minimax(game.board, player, depth - 1, alpha, beta, False))
                    game.loadState(board)
                    alpha = max(alpha, value)
                    if alpha >= beta:
                        break  # beta prune
            if alpha >= beta:
                break
    else:
        value = float('inf')
        for i in range(4):
            for j in range(4):
                if game.heights[i, j] < 4:
                    game.move(i, j, player)
                    value = min(value, minimax(game.board, player, depth - 1, alpha, beta, True))
                    game.loadState(board)
                    beta = min(beta, value)
                    if alpha >= beta:
                        break  # alpha prune
            if alpha >= beta:
                break
    return value

def findBestMove(board, player, depth):
    game = TicTacToe3D().loadState(board)
    best = float('-inf')
    bestMove = None
    possible_moves = [(i, j) for i in range(4) for j in range(4) if game.heights[i, j] < 4]

    if not possible_moves:
        return None
    
    bestMove = random.choice(possible_moves)
                      
    for i, j in possible_moves:
        game.move(i, j, player)
        new_board = game.board
        # print(new_board)
        # print(new_board)
        # Using alpha-beta pruning in minimax call
        moveVal = minimax(new_board, player, depth, float('-inf'), float('inf'), False)
        game.loadState(board)
        if moveVal > best:
            best = moveVal
            bestMove = (i, j)
        print(f"Move value for ({i}, {j}): {moveVal}")  # To observe the value of each move
    return bestMove

dummy_board = np.zeros((4, 4, 4), dtype=int)
# floor, row, col
dummy_board[0, 0, 0] = 1
dummy_board[1, 1, 1] = 1
dummy_board[2, 2, 2] = 1
# dummy_board[0, 1, 1] = 1
dummy_board[0, 3, 3] = -1
dummy_board[1, 3, 3] = -1
# dummy_board[2, 3, 3] = -1
# print(dummy_board)
x = findBestMove(dummy_board, 1, 1)
print(x)


Move value for (0, 0): 13
Move value for (0, 1): 12
Move value for (0, 2): 13
Move value for (0, 3): 17
Move value for (1, 0): 12
Move value for (1, 1): 16
Move value for (1, 2): 14
Move value for (1, 3): 14
Move value for (2, 0): 13
Move value for (2, 1): 14
Move value for (2, 2): 14
Move value for (2, 3): 14
Move value for (3, 0): 17
Move value for (3, 1): 14
Move value for (3, 2): 14
Move value for (3, 3): 14
(0, 3)


In [92]:
# def count_wins(board, player):
#     global all_wins
#     c = 0
#     for win in all_wins:
#         if np.sum(win * board) == 4 * player:
#             c += 1
#     return c

# dummy_board = np.ones((4, 4, 4), dtype=int)
# print(count_wins(dummy_board, 1))